# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2023
topic: str = "FARMACIA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [4]:
add_list: list = [x for x in range(19, 41, 1)] + [x for x in range(48, 82, 1)] + [123, 125] + [x for x in range(127, 150, 2)] + [x for x in range(183, 189, 1)] + [x for x in range(194, 203, 1)] + [x for x in range(205, 211, 1)]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,1. Teniendo en cuenta la relación estructura-
1,actividad ge neral de los agentes ne urolépticos
2,"derivados de butirofenona, ¿cuál de las modifi-"
3,caciones pr opuestas c onduciría a compuestos
4,más activos?:
5,1. Sustitución del grupo carbonilo B por un
6,tiocarbonilo.
7,2. Eliminación del átomo de flúor en el ani-
8,llo aromático A.
9,"3. Introducción de un an illo d e piperidina,"


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [6]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,3.0,43,2.0,85,2.0,127,3.0,169,3.0
1,2,2.0,44,1.0,86,3.0,128,3.0,170,2.0
2,3,4.0,45,3.0,87,2.0,129,1.0,171,1.0
3,4,3.0,46,2.0,88,2.0,130,2.0,172,4.0
4,5,2.0,47,NaN,89,1.0,131,4.0,173,3.0
5,6,1.0,48,2.0,90,3.0,132,NaN,174,2.0
6,7,1.0,49,4.0,91,3.0,133,3.0,175,2.0
7,8,4.0,50,3.0,92,3.0,134,2.0,176,4.0
8,9,4.0,51,4.0,93,3.0,135,1.0,177,2.0
9,10,2.0,52,3.0,94,1.0,136,1.0,178,4.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [7]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [8]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [9]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [10]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [11]:
raw_exam_df.head(10)

0       1. Teniendo en cuenta la relación estructura-
1    actividad ge neral de los agentes ne urolépticos
2     derivados de butirofenona, ¿cuál de las modifi-
3       caciones pr opuestas c onduciría a compuestos
4                                       más activos?:
5         1. Sustitución del grupo carbonilo B por un
6                                       tiocarbonilo.
7        2. Eliminación del átomo de flúor en el ani-
8                                    llo aromático A.
9       3. Introducción de un an illo d e piperidina,
Name: text, dtype: object

In [12]:
raw_exam_df.tail(10)

2325                                2. Ceftriaxona.
2326                               3. Azitromicina.
2327                                4. Doxiciclina.
2328           210. El voriconazol tiene actividad:
2329    1. Fungiestática frente a hongos filamento-
2330                                           sos.
2331             2. Fungicida frente a Candida spp.
2332    3. Fungicida frente a Cryptococcus neof or-
2333                                          mans.
2334         4. Fungicida frente a Aspergillus spp.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [13]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [14]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Teniendo en cuenta la relación estructuraac...
1    1. Sustitución del grupo carbonilo B por un ti...
2    2. Eliminación del átomo de flúor en el anillo...
3    3. Introducción de un an illo d e piperidina, ...
4                      4. Alargamiento de la cadena C.
5    2. La siguiente reacción es un proceso de semi...
6                                          1. Adenina.
7                                          2. Guanina.
8                                3. 8-amino-9H-purina.
9                            4. 2-amino-9H-purin-8-ol.
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [15]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 11
Number of rows not ending with '.', ':' or '?' = 10
[84, 571, 593, 676, 690, 896, 897, 918, 969, 974]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [16]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [17]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [18]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [84, 335, 571, 593, 676, 690, 896, 897, 918, 969, 974]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 84:
82            2. Sustitución del carbono 2 por oxígeno.
83                       3. Aromatización del anillo A.
84        4. Introducción de u n metilo en la p osición
85                                                 16α.
86    18. ¿Cuál de las siguientes estrategias aument...
Name: text, dtype: object
Id to fix 571:
569    2. Vancomicina - área bajo la curva (AUC)/CMI ...
570    3. Ceftazidima/avibactam - c oncentración pico...
571           4. Fosfomicina - concentración valle entre
572           20-40 mcg/mL para determinar su seguridad.
573    115. AM va a i niciar t ratamiento con c apeci...
Name: text, dtype: object
Id to fix 593:
591                                      3. Endocitosis.
592                                   4. Pares de iones.
593    119. Se admi nistra un antibiótico medi ante i...
594    25% del peso corporal. L a Css (c oncentración...
595                                           1. 250 mg.
Name: text, dtype: object
Id to fix 676:
674    135. E

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [19]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [20]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [21]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [22]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [23]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [24]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. Teniendo en cuenta la relación estructuraac...,1. Sustitución del grupo carbonilo B por un ti...,2. Eliminación del átomo de flúor en el anillo...,"3. Introducción de un an illo d e piperidina, ...",4. Alargamiento de la cadena C.
1,2. La siguiente reacción es un proceso de semi...,1. Adenina.,2. Guanina.,3. 8-amino-9H-purina.,4. 2-amino-9H-purin-8-ol.
2,3. ¿Qué representa el símbolo 𝝅 (pi) en una ec...,1. El co eficiente d e partición octanol/agua ...,2. Los efectos electrónicos de un sustituyente.,3. Los efectos estéricos de un sustituyente.,4. La constante de hidrofobicidad de un sustit...
3,4. Un compuesto inactivo que necesita ser acti...,1. Fármaco huérfano.,2. Fármaco bioequivalente.,3. Profármaco.,4. Antimetabolito.
4,5. ¿Cuál de l os siguientes grupos funcionales...,1. Grupo nitrilo.,2. Tetrazol.,3. Tiofeno.,4. Fenol.
5,6. ¿Qué efec to tiene en l a actividad de l a...,1. Conduce a compuestos que se el iminan más ...,2. Conduce a compuestos de vida media más larga.,3. Conduce a c ompuestos ant agonistas del GABA.,4. Conduce a c ompuestos sin actividad a nsio...
6,7. La conjugación con aminoácidos es un a re ...,1. Ácidos carboxílicos activados como tioésteres.,2. Aminas primarias aromáticas.,3. Fenoles.,"4. Hidroxilaminas ar omáticas, procedentes de ..."
7,8. La hidrólisis de la amida en la po sición 7...,1. Enamina.,2. Cloruro de ácido.,3. Hemiacetal.,4. Iminoéter.
8,9. La siguiente estruc tura corresponde al ant...,1. Apertura del anillo de aziridina.,2. Hidrólisis del carbamato.,3. Formación de una enamina por pérdida de met...,4. Reducción del anillo de quinona.
9,10. ¿Qué relación tiene el sigui ente c ompues...,1. Es un análogo que establece interacciones a...,2. Es un profármaco más lipófilo para su uso e...,3. Es un anál ogo blando que s e metaboliza po...,4. Es un anál ogo de acción c orta porque la...


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,3.0
1,2,2.0
2,3,4.0
3,4,3.0
4,5,2.0
5,6,1.0
6,7,1.0
7,8,4.0
8,9,4.0
9,10,2.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. Teniendo en cuenta la relación estructuraac...,1. Sustitución del grupo carbonilo B por un ti...,2. Eliminación del átomo de flúor en el anillo...,"3. Introducción de un an illo d e piperidina, ...",4. Alargamiento de la cadena C.,3.0
1,2. La siguiente reacción es un proceso de semi...,1. Adenina.,2. Guanina.,3. 8-amino-9H-purina.,4. 2-amino-9H-purin-8-ol.,2.0
2,3. ¿Qué representa el símbolo 𝝅 (pi) en una ec...,1. El co eficiente d e partición octanol/agua ...,2. Los efectos electrónicos de un sustituyente.,3. Los efectos estéricos de un sustituyente.,4. La constante de hidrofobicidad de un sustit...,4.0
3,4. Un compuesto inactivo que necesita ser acti...,1. Fármaco huérfano.,2. Fármaco bioequivalente.,3. Profármaco.,4. Antimetabolito.,3.0
4,5. ¿Cuál de l os siguientes grupos funcionales...,1. Grupo nitrilo.,2. Tetrazol.,3. Tiofeno.,4. Fenol.,2.0
5,6. ¿Qué efec to tiene en l a actividad de l a...,1. Conduce a compuestos que se el iminan más ...,2. Conduce a compuestos de vida media más larga.,3. Conduce a c ompuestos ant agonistas del GABA.,4. Conduce a c ompuestos sin actividad a nsio...,1.0
6,7. La conjugación con aminoácidos es un a re ...,1. Ácidos carboxílicos activados como tioésteres.,2. Aminas primarias aromáticas.,3. Fenoles.,"4. Hidroxilaminas ar omáticas, procedentes de ...",1.0
7,8. La hidrólisis de la amida en la po sición 7...,1. Enamina.,2. Cloruro de ácido.,3. Hemiacetal.,4. Iminoéter.,4.0
8,9. La siguiente estruc tura corresponde al ant...,1. Apertura del anillo de aziridina.,2. Hidrólisis del carbamato.,3. Formación de una enamina por pérdida de met...,4. Reducción del anillo de quinona.,4.0
9,10. ¿Qué relación tiene el sigui ente c ompues...,1. Es un análogo que establece interacciones a...,2. Es un profármaco más lipófilo para su uso e...,3. Es un anál ogo blando que s e metaboliza po...,4. Es un anál ogo de acción c orta porque la...,2.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. Teniendo en cuenta la relación estructuraac...,1. Sustitución del grupo carbonilo B por un ti...,2. Eliminación del átomo de flúor en el anillo...,"3. Introducción de un an illo d e piperidina, ...",4. Alargamiento de la cadena C.,3,2023
1,2. La siguiente reacción es un proceso de semi...,1. Adenina.,2. Guanina.,3. 8-amino-9H-purina.,4. 2-amino-9H-purin-8-ol.,2,2023
2,3. ¿Qué representa el símbolo 𝝅 (pi) en una ec...,1. El co eficiente d e partición octanol/agua ...,2. Los efectos electrónicos de un sustituyente.,3. Los efectos estéricos de un sustituyente.,4. La constante de hidrofobicidad de un sustit...,4,2023
3,4. Un compuesto inactivo que necesita ser acti...,1. Fármaco huérfano.,2. Fármaco bioequivalente.,3. Profármaco.,4. Antimetabolito.,3,2023
4,5. ¿Cuál de l os siguientes grupos funcionales...,1. Grupo nitrilo.,2. Tetrazol.,3. Tiofeno.,4. Fenol.,2,2023
5,6. ¿Qué efec to tiene en l a actividad de l a...,1. Conduce a compuestos que se el iminan más ...,2. Conduce a compuestos de vida media más larga.,3. Conduce a c ompuestos ant agonistas del GABA.,4. Conduce a c ompuestos sin actividad a nsio...,1,2023
6,7. La conjugación con aminoácidos es un a re ...,1. Ácidos carboxílicos activados como tioésteres.,2. Aminas primarias aromáticas.,3. Fenoles.,"4. Hidroxilaminas ar omáticas, procedentes de ...",1,2023
7,8. La hidrólisis de la amida en la po sición 7...,1. Enamina.,2. Cloruro de ácido.,3. Hemiacetal.,4. Iminoéter.,4,2023
8,9. La siguiente estruc tura corresponde al ant...,1. Apertura del anillo de aziridina.,2. Hidrólisis del carbamato.,3. Formación de una enamina por pérdida de met...,4. Reducción del anillo de quinona.,4,2023
9,10. ¿Qué relación tiene el sigui ente c ompues...,1. Es un análogo que establece interacciones a...,2. Es un profármaco más lipófilo para su uso e...,3. Es un anál ogo blando que s e metaboliza po...,4. Es un anál ogo de acción c orta porque la...,2,2023


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [34]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 91
Number of rows in filtered df = 91


**2. Showing the filtered DataFrame**

Using head and tail

In [35]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,19. ¿Cuál de los siguientes antibióticos betal...,1. Cefepime.,2. Ceftolozano-tazobactam.,3. Meropenem.,4. Ertapenem.,4,2023
1,"20. Para generar una infecci ón, los derm ato...",1. Hemoglobina.,2. Colágeno.,3. Queratina.,4. Ácido hialurónico.,3,2023
2,21. El marcador 1- 3-beta-D-glucano se ele va ...,1. Aspergilosis invasiva.,2. Candidiasis hepatoesplénica.,3. Mucormicosis rinocerebral.,4. Neumonía causad a por Pn eumocystis jirovecii.,3,2023
3,22. ¿Cuál de las siguientes c aracterísticas c...,1. Es un microorganismo cuyo reservorio es exc...,2. Es un bacilo grampositivo.,3. Produce dia rrea asociada al consumo de car...,4. En la actualidad se dispone de una vacuna e...,3,2023
4,23. ¿Cuál de las infecciones causadas por uno ...,1. Treponema pallidum.,2. Streptococcus pyogenes.,3. Mycoplasma pneumoniae.,4. Streptococcus agalactiae.,3,2023
5,"24. En rela ción con la sífilis, seña le la re...",1. El tratamiento de elección es colistina.,2. El age nte etiológico es Treponema pallidum.,3. Es una infección de transmisión sexual.,4. En fase avanzada puede afectar al sistema n...,1,2023
6,25. En el cultivo de una muestra de l íquido c...,1. Streptococcus pneumoniae.,2. Klebsiella pneumoniae.,3. Escherichia coli.,4. Staphylococcus aureus.,1,2023
7,26. Señale la afirmación INCORRECTA respecto a...,1. Es un bacilo grampositivo f ormador de espo...,2. Es anaerobio obligado.,3. El tratamiento de elección de las infeccio...,4. Existen cepas epidémicas que han causado br...,3,2023
8,27. ¿Cuál de l as siguientes r espuestas es c ...,1. Es un virus A RN bi catenario de simetría h...,2. Reconoce el receptor humano ACE2 (enzima co...,3. El gen estructural S es idéntico al del vir...,4. Pertenece a la familia Orthomyxoviridae.,2,2023
9,28. ¿Cuál de las siguientes especies NO perten...,1. Enterobacter asburiae.,2. Enterobacter bugandensis.,3. Enterobacter kobei.,4. Enterobacter hormaechei.,0,2023


In [36]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
81,199. Indique la re lación I NCORRECTA entre qu...,1. CXCL13 contribuye a l a entrada de l os li...,2. CXCL5 dirige el movimiento de l os l infoci...,3. CXCL8 es una quimiocina i mplicada e n la q...,4. CCR7 participa en la mig ración d e los lin...,2,2023
82,200. El componente farmacológico de ARN monoca...,1. TLR2 y TLR4.,2. TLR1 y TLR5.,3. TLR7 y TLR8.,4. TLR9.,3,2023
83,201. En cuanto a la artritis reumatoide:,1. Una d e las pri ncipales cito cinas imp lic...,2. Las articulaciones más comúnmente afectadas...,3. Para d iagnosticar u na artritis reumatoide...,4. Es indiferente para el pronóstico de la enf...,3,2023
84,202. ¿Cuál de l as si guientes re spuestas es ...,1. Uno de los síntomas de la intoxicación es l...,"2. La exc reción del cadmio es muy lenta, por...",3. El principal órg ano d iana d e la intoxica...,4. Puede t ratarse con fá rmacos quelantes com...,3,2023
85,205. Los huesecillos de la cavidad del o ído m...,1. Al conducto endolinfático.,2. Al conducto coclear.,3. Al helicotrema.,4. A la ventana oval.,4,2023
86,206. ¿Cuál de l as siguientes hormonas atravie...,1. Somatostatina.,2. Prostaglandina E.,3. Tiroxina.,4. Óxido nítrico.,3,2023
87,207. ¿Cuál es la asociación correcta entre hor...,1. Aldosterona – zona fasciculada.,2. Dehidroepiandrosterona (DHEA) – zona estriada.,3. Cortisol – zona reticular.,4. Desoxicorticosterona – zona glomerular.,4,2023
88,208. ¿Qué ocurre cuando aumenta la prevalencia...,1. Que el valor pred ictivo po sitivo d e la p...,2. Que disminuyen en valor absoluto los falsos...,3. Que el valor predictivo negativo de la pru...,4. No quedan afectados los valores predictivos...,3,2023
89,209. ¿Cuál es el tratamiento de primera línea ...,1. Penicilina G.,2. Ceftriaxona.,3. Azitromicina.,4. Doxiciclina.,1,2023
90,210. El voriconazol tiene actividad:,1. Fungiestática frente a hongos filamentosos.,2. Fungicida frente a Candida spp.,3. Fungicida frente a Cryptococcus neof ormans.,4. Fungicida frente a Aspergillus spp.,4,2023


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [37]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [38]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [39]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [40]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()